**CSC 466: Knowledge Discovery in Data**

**Individual Test**

**Task 3**

**Your Name :**

**Cal Poly Email:**


**Your Assignment**:

1. Complete the code for the Naive Bayes Classifier
2. Complete the training and testing of the Classifier
3. Compute the accuracy of the classifier and output the overall accuracy and the confusion matrix.

In [9]:
## Imports

import numpy as np
from matplotlib import pyplot as plt
import seaborn
%matplotlib inline

**estimateNBModel**: function to produce all parameter estimates for the Naive Bayes model.

This function returns two structures: an array of class probabilities P(Class = c_i),
and a collection of 
$$P(A_j = a_k|Class = c_i)$$
probability estimators for the probability that an observed record from class $c_i$ will take the value $a_k$ for its $j$th attribute $A_j$.

The latter collection is stored as a list model[0..nClasses-1], where each model[i] is itself a list of length nAttributes of probability distributions over the values of each attribute.


Input parameters for estimateNBModel():

data: training set data points

labels: labels for the training set data

nAttributes: number of attributes in the dataset

attributeRanges: number of unique values each attribute takes

nClasses: number of classes (class labels)


**Your Task** :   write the getClassProbability() function that given
   
   * the list of labels (labels) of the training set

   * the class label (classId),
   
   * and the total number of classes in the dataset (nClasses)
   
 returns the probability estimate for $P(Class = classId)$

In [3]:
import pandas as pd

In [17]:
def getClassProbability(labels, classId,nClasses):
    return (labels == classId).sum() / len(labels)

**Your Task**: write the getNBEstimate() function that given

* the training set (data)

* the class labels for the training set (labels)

* the class label for which the estimate is being produced (classLabel)

* the attribute for which the estimate is being produced (attId)

* the attribute value for which the estimate is being produced (attValue)

* and the total number of values attribute attId has (nValues)

produces the probability estimate for $P(A_{attId} = attValue | Class = classLabel)$

In [23]:
def getNBEstimate(data, labels, classLabel, attId, attValue, nValues):
    return ((data[: ,attId] == attValue) * (labels == classLabel)).sum() / (labels == classLabel).sum()

In [19]:
def estimateNBModel(data, labels, nAttributes, attributeRanges, nClasses ):
    ## Naive Bayes Model consists of two types of estimators
    
    ## First, we estimate the probability of seeing an object from a specific class
    
    classProbabilities = [getClassProbability(labels,l, nClasses) for l in range(nClasses)]
    
    ## now we estimate the probabilities of seeing a specific value of a specific attribute in
    ## a data point from a given class
    
    ## for each class create the appropriate estimates
    model = []                  # model is the list of estimates for all classes
    for i in range(nClasses):   # for each class
        ## for each attribute
        classDistr = []         # classDistr is the collection of estimates for one class
        for j in range(nAttributes):
            estimates = []                    # estimates is a distribution of estimates for a single attribute
            for k in range(attributeRanges[j]):
                est = getNBEstimate(data, labels, i,j,k, attributeRanges[j])
                estimates.append(est)
            classDistr.append(estimates)
        model.append(classDistr)
    
    return classProbabilities, model
    

**Predicting The Class**

function predictNBLabels() predicts the class labels for all data points in the test set.

function predictNB() computes the probability estimates for each class and selects the class with the highest estimate for a single data point

function predictNBClass computes the probability estimate for a specific class.

**Your Task**: implement predictNBClass()

its parameters are:

* point: the data point for which the estimate is given

* classProb:  the class probability P(Class = classID) for the class 

* classModel: the portion of the Naive Bayes model related to predicting this particular class

(note that class label is not passed, but all proper values are selected in predictNB())

In [38]:
def predictNBClass(point, classProb, classModel):
    jointProb = 1
    for attr, val in enumerate(point):
        jointProb = jointProb * classModel[int(attr)][int(val)]
    return classProb * jointProb

predictNB()  and predictNBLabels() parameters

point: data point

classProbabilities: the list of $P(Class = c_i)$ estimates

model: the collection of $P(A_j = a_k |Class = c_i)$ probability estimates

nClasses: number of class labels in the dataset

You do not need to touch this code

In [31]:
def predictNB(point, classProbabilities, model,nClasses):
    
    predictions= np.array([predictNBClass(point, classProbabilities[i],model[i]) for i in range(nClasses)])
    
    predictedClass = np.argmax(predictions)
    return predictedClass

    
    
    

In [32]:
def predictNBLabels(data, classProbabilities, model, nClasses):
    predicted = [predictNB(point, classProbabilities, model, nClasses) for point in data]
    return predicted

** Load Data **

In [33]:
filename="data8.csv"

rawData = np.loadtxt(filename, delimiter = ",")

## let's keep only the two columns with the data attributes

nAttributes = rawData.shape[1] - 1

data = rawData[:,0:nAttributes]
labels = rawData[:,nAttributes]

In [34]:
rawData

array([[2., 0., 2., 3., 0., 0.],
       [0., 1., 0., 1., 2., 2.],
       [2., 0., 0., 3., 3., 2.],
       ...,
       [1., 2., 1., 1., 0., 0.],
       [3., 0., 2., 3., 3., 1.],
       [2., 0., 2., 1., 1., 1.]])

** Train the Model**

In the cells below the entire dataset  is used to train the model. 

This allows us to see the predictions, but this is not a fair way to evaluate the quality of prediction.

In [35]:
### Let us compute how many unique values each attribute has.
### all attributes have values 0,1,.., k-1 where k is the number of unique values for that attribute.

attributeRanges= [np.unique(data[:,i]).shape[0] for i in range(nAttributes)]

## number of classes

nClasses = np.unique(labels).shape[0]

In [36]:
d,m = estimateNBModel(data,labels,nAttributes, attributeRanges,nClasses)

In [39]:
predicted = predictNBLabels(data,d,m,nClasses)

In [ ]:
An easy way to see where we missed:

In [40]:
predicted - labels

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -2.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,
       -2.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1., -2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,
        0.,  0., -2.,  1.,  0.,  0.,  1., -1.,  0.,  0.,  0.,  0., -1.,
        0., -1.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -2.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0., -2.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
       -2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1

**Your Task**:

Split your dataset into the training set (2/3 of the data = 200 data points) and the test set (the remaining 1/3 of the data points = 100 points). Select your points at random, but make it reproducible by setting the seed.

In [57]:
split_3 = np.array_split(pd.DataFrame(data).sample(frac=1), 3)
train = pd.concat(split_3[0:2])
train_idx = list(train.index)
train = train.to_numpy()
test = split_3[1]
test_idx = list(test.index)
test = test.to_numpy()

C:\Users\hsu_m\anaconda3\envs\csc466\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


** Your Task**: train the model on the training set

In [58]:
d_train, m_train = estimateNBModel(train,labels[train_idx],nAttributes,attributeRanges,nClasses)

** Your Task**: evaluate the model on the test set. Retrieve the predicted labels for the test set data points.

In [62]:
test_pred = predictNBLabels(test,d,m,nClasses)
test_pred = np.array(test_pred)
test_pred

array([0, 2, 0, 1, 2, 1, 2, 0, 1, 1, 2, 0, 1, 2, 2, 0, 0, 2, 1, 2, 2, 2,
       1, 1, 0, 2, 2, 0, 0, 0, 1, 2, 2, 1, 0, 0, 2, 2, 0, 1, 0, 1, 0, 1,
       0, 0, 1, 2, 0, 2, 1, 0, 0, 2, 2, 0, 2, 0, 1, 2, 0, 1, 2, 1, 1, 1,
       2, 2, 0, 2, 0, 1, 2, 2, 1, 0, 0, 2, 0, 1, 1, 2, 2, 0, 0, 1, 1, 2,
       1, 2, 1, 0, 0, 0, 0, 2, 0, 0, 2, 2], dtype=int64)

** Your Task**: compute the predictive accuracy on the training set and report it. 

Compute the predictive accuract on the test set and report it.

Is there any evidence that the model overfits? (put a note in a markdown cell0


In [67]:
(test_pred == labels[test_idx]).sum() / len(test_pred)

0.83

In [69]:
train_pred = predictNBLabels(train,d,m,nClasses)
train_pred = np.array(train_pred)
(train_pred == labels[train_idx]).sum() / len(train_pred)

0.835

Answer the overfit question here.

**Your Task**: compute and output the confusion matrix

In [70]:
pd.crosstab(pd.Series(test_pred, name="pred"), pd.Series(labels[test_idx], name="obs"))

obs,0.0,1.0,2.0
pred,,,
0,30,1,5
1,2,25,1
2,2,6,28


**Congratulations!** Your are done.

Download the notebook, and submit it using the 

        handin dekhtyar 446-test <file>
 command.